In [1]:
import flask
import numpy as np
import tensorflow as tf
from keras.models import load_model

Using TensorFlow backend.


In [2]:
# initialize our Flask application and the Keras model
app = flask.Flask(__name__)
def init():
    global model,graph
    # load the pre-trained Keras model
    model = load_model('model.SSL')
    graph = tf.get_default_graph()

In [7]:
@app.route('/imageclassifier/predict/', methods=['POST'])
def image_classifier():
    # Decoding and pre-processing base64 image
    img = image.img_to_array(image.load_img(BytesIO(base64.b64decode(request.form['b64'])),
                                            target_size=(224, 224))) / 255.

    img = np.expand_dims(img, axis = 0)
    # this line is added because of a bug in tf_serving(1.10.0-dev)
    img = img.astype('float16')

    # Creating payload for TensorFlow serving request
    #payload = {
    #    "instances": [{'input_image': img.tolist()}]
    #}

    # Making POST request
    r = requests.post('http://localhost:9000/v1/models/ImageClassifier:predict', json=payload)

    # Decoding results from TensorFlow Serving server
    # pred = json.loads(r.content.decode('utf-8'))
    with graph.as_default():
        pred = model.predict(img)[0][0]
    

    # Returning JSON response to the frontend
    return jsonify((np.array(pred['predictions']))[0])

AssertionError: View function mapping is overwriting an existing endpoint function: image_classifier